In [1]:
%load_ext autoreload
%autoreload 2
main_dir = r'../../..'
data_path = main_dir+'/data'
import sys
sys.path.append(main_dir)
from typing import List, Set, Dict, Optional, Any, Tuple, Type, Union

from BayDS.lib.pipeline import *
from BayDS.lib.io import *
import os


In [2]:
data_dir = f'{main_dir}/Snapshots/1/Test'
p = Pipeline(working_folder=f'{main_dir}/Snapshots/1/DynamicsKeras')

p.add_node(LoaderNode, None, 'data',
           params={
               'input_directory': data_dir,
               'file': 'label_encoded_data.pkl'
           })
p.add_node(LoaderNode, None, 'oof',
           params={
               'input_directory': main_dir+'/Snapshots/Dynamic',
               'file': 'oof_best_submit.csv'
           })
p.add_node(LoaderNode, None, 'predictions',
           params={
               'input_directory': main_dir+'/Snapshots/Dynamic',
               'file': 'ieee_with_additions.csv'
           })
p.run(verbose=True)
p.save()

---------------------------
0: LoaderNode [2019-09-22 22:42:15]
params:
 {'input_directory': '../../../Snapshots/1/Test', 'file': 'label_encoded_data.pkl'}
---------------------------
1: LoaderNode [2019-09-22 22:42:18]
params:
 {'input_directory': '../../../Snapshots/Dynamic', 'file': 'oof_best_submit.csv'}
---------------------------
2: LoaderNode [2019-09-22 22:42:19]
params:
 {'input_directory': '../../../Snapshots/Dynamic', 'file': 'ieee_with_additions.csv'}


In [3]:
df = p.data['data']
df = df[['isFraud', 'new_card_id']]

In [4]:
pred = pd.concat([p.data['oof'].set_index('TransactionID')['score'],p.data['predictions'].set_index('TransactionID')['isFraud']],axis=0)#
pred.index

Int64Index([2987000, 2987001, 2987002, 2987003, 2987004, 2987005, 2987006,
            2987007, 2987008, 2987009,
            ...
            4170230, 4170231, 4170232, 4170233, 4170234, 4170235, 4170236,
            4170237, 4170238, 4170239],
           dtype='int64', name='TransactionID', length=1097231)

In [5]:
pred

TransactionID
2987000    0.013343
2987001    0.008355
2987002    0.003535
2987003    0.001327
2987004    0.002611
             ...   
4170235    0.003432
4170236    0.002554
4170237    0.003562
4170238    0.003217
4170239    0.002311
Length: 1097231, dtype: float64

In [6]:
df['pred_0'] = pred

c:\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [7]:
df

,isFraud,new_card_id,pred_0
TransactionID,,,
2987000,0,130935,0.013343
2987001,0,47322,0.008355
2987002,0,40076,0.003535
2987003,0,9790,0.001327
2987004,0,113643,0.002611
...,...,...,...
4170235,-1,21517,0.003432
4170236,-1,9161,0.002554
4170237,-1,48339,0.003562


In [8]:
test = df[['pred_0','new_card_id']].groupby('new_card_id').shift()

In [9]:
shifts = {}
for shift in range(-20,20):
    s = df[['pred_0','new_card_id']].groupby('new_card_id').shift(shift)['pred_0']
    shifts[shift] = s

In [10]:
for k,v in shifts.items():
    if k!=0:
        df[f'pred_{k}']=v

c:\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
X = df[df.isFraud>=0].drop(['isFraud'],axis=1)
y = df[df.isFraud>=0]['isFraud']
test = df[df.isFraud<0].drop(['isFraud'],axis=1)


In [13]:
from category_encoders.cat_boost import CatBoostEncoder

encoder = CatBoostEncoder(verbose=1, cols=['new_card_id'])
encoder.fit(X, y)

X = encoder.transform(X)
test = encoder.transform(test)



In [14]:
X

,new_card_id,pred_0,pred_-20,pred_-19,pred_-18,pred_-17,pred_-16,pred_-15,pred_-14,pred_-13,...,pred_10,pred_11,pred_12,pred_13,pred_14,pred_15,pred_16,pred_17,pred_18,pred_19
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0.034990,0.013343,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987001,0.030910,0.008355,0.018132,0.000677,0.003601,0.002565,0.456456,0.117105,0.040508,0.024899,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987002,0.015724,0.003535,0.000534,0.001498,0.020098,0.007489,0.001951,0.008487,0.003347,0.001273,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987003,0.000265,0.001327,0.001506,0.001295,0.003040,0.005204,0.005135,0.006308,0.002614,0.001262,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987004,0.147856,0.002611,NaN,NaN,NaN,NaN,NaN,0.016008,0.000480,0.000690,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3577535,0.034990,0.002267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3577536,0.006998,0.004644,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3577537,0.038284,0.003130,0.003385,0.763504,0.164259,0.005626,0.000096,0.002307,0.005009,0.004402,...,0.006830,0.003486,0.004347,0.005189,0.008216,0.003562,0.041620,0.032738,0.013255,0.006538


In [15]:
#Keras   strart
from sklearn.preprocessing import StandardScaler

In [16]:
for column in X.columns:
    print(column)
    col = pd.concat([X[column],test[column]])
    filtered = col[~col.isin([np.inf, -np.inf])].dropna()
    vmax = filtered.max()
    vmin = filtered.min()
    vmean = filtered.mean()
    col = col.replace({
        np.inf:vmax,
        -np.inf:vmin
       }).fillna(-1)
    if vmax > 100 and vmin >= 0:
        col = np.log1p(col)
    
    scaler = StandardScaler()
    col.values[...] = scaler.fit_transform(col.values.reshape(-1,1)).flatten()
    
    X[column] = col[X.index]
    test[column] = col[test.index]
    
    

new_card_id
pred_0
pred_-20
pred_-19
pred_-18
pred_-17
pred_-16
pred_-15
pred_-14
pred_-13
pred_-12
pred_-11
pred_-10
pred_-9
pred_-8
pred_-7
pred_-6
pred_-5
pred_-4
pred_-3
pred_-2
pred_-1
pred_1
pred_2
pred_3
pred_4
pred_5
pred_6
pred_7
pred_8
pred_9
pred_10
pred_11
pred_12
pred_13
pred_14
pred_15
pred_16
pred_17
pred_18
pred_19


In [17]:
X

,new_card_id,pred_0,pred_-20,pred_-19,pred_-18,pred_-17,pred_-16,pred_-15,pred_-14,pred_-13,...,pred_10,pred_11,pred_12,pred_13,pred_14,pred_15,pred_16,pred_17,pred_18,pred_19
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,-0.004654,-0.097789,-0.854498,-0.865575,-0.877646,-0.890946,-0.905609,-0.921882,-0.940055,-0.960388,...,-1.041028,-1.010161,-0.983543,-0.960376,-0.940032,-0.921857,-0.905595,-0.890938,-0.877646,-0.865579
2987001,-0.052361,-0.141396,1.103609,1.055995,1.046629,1.028610,1.879336,1.211807,1.045562,0.994377,...,-1.041028,-1.010161,-0.983543,-0.960376,-0.940032,-0.921857,-0.905595,-0.890938,-0.877646,-0.865579
2987002,-0.229947,-0.183534,1.069764,1.057572,1.078260,1.038037,1.010259,1.004345,0.974647,0.949317,...,-1.041028,-1.010161,-0.983543,-0.960376,-0.940032,-0.921857,-0.905595,-0.890938,-0.877646,-0.865579
2987003,-0.410723,-0.202837,1.071634,1.057181,1.045553,1.033662,1.016348,1.000182,0.973249,0.949296,...,-1.041028,-1.010161,-0.983543,-0.960376,-0.940032,-0.921857,-0.905595,-0.890938,-0.877646,-0.865579
2987004,1.315183,-0.191614,-0.854498,-0.865575,-0.877646,-0.890946,-0.905609,1.018710,0.969177,0.948204,...,-1.041028,-1.010161,-0.983543,-0.960376,-0.940032,-0.921857,-0.905595,-0.890938,-0.877646,-0.865579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3577535,-0.004654,-0.194622,-0.854498,-0.865575,-0.877646,-0.890946,-0.905609,-0.921882,-0.940055,-0.960388,...,-1.041028,-1.010161,-0.983543,-0.960376,-0.940032,-0.921857,-0.905595,-0.890938,-0.877646,-0.865579
3577536,-0.331989,-0.173845,-0.854498,-0.865575,-0.877646,-0.890946,-0.905609,-0.921882,-0.940055,-0.960388,...,-1.041028,-1.010161,-0.983543,-0.960376,-0.940032,-0.921857,-0.905595,-0.890938,-0.877646,-0.865579
3577537,0.033864,-0.187079,1.075247,2.520829,1.354671,1.034470,1.006714,0.992541,0.977819,0.955284,...,0.881484,0.903221,0.930403,0.955353,0.982456,0.993447,1.084601,1.084904,1.063724,1.065855


In [18]:
from sklearn.model_selection import KFold
n_fold = 5
# folds = TimeSeriesSplit(n_splits=n_fold)
folds = KFold(n_splits=5)

In [19]:
from BayDS.lib.training import *

Using TensorFlow backend.
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as

In [22]:
def NNModel_maker():
    k.clear_session()
    
#     categorical_inputs = []
#     for cat in categorical:
#         categorical_inputs.append(Input(shape=[1], name=cat))

#     categorical_embeddings = []
#     for i, cat in enumerate(categorical):
#         categorical_embeddings.append(
#             Embedding(category_counts[cat], int(np.log1p(category_counts[cat]) + 1), name = cat + \
#                       "_embed")(categorical_inputs[i]))

#     categorical_logits = Concatenate(name = "categorical_conc")([Flatten()(SpatialDropout1D(.1)(cat_emb)) for cat_emb in categorical_embeddings])
# 
    numerical_inputs = Input(shape=[X.shape[1]], name = 'all')
    numerical_logits = Dropout(.3)(numerical_inputs)
  
    x = numerical_logits

    x = Dense(150, activation = 'relu')(x)
    x = Dropout(.3)(x)
    x = Dense(70, activation = 'relu')(x)
    x = Dropout(.3)(x)
    out = Dense(1, activation = 'sigmoid')(x)    

    model = Model(inputs= [numerical_inputs],outputs=out)
    loss = "binary_crossentropy"
    model.compile(optimizer=Adam(lr = 0.0003), loss = loss)
    return model


params = {
    'batch_size': 8000,
    'epochs': 130,
    'verbose': True,
         }
train_options = {
    "model_type":'keras',
    "params": params,
    "eval_metric":'auc',
    'averaging': 'usual',
    'use_groups': False,
    'fold_name': folds.__class__.__name__,
    'n_splits': n_fold
   
}

In [23]:
gc.collect()
result_dict_keras = train_model_classification(model=NNModel_maker, 
                                             X=X,
                                             X_test=test,
                                             y=y, params=params, folds=folds,
                                             model_type=train_options['model_type'], 
                                             eval_metric=train_options['eval_metric'],
                                             plot_feature_importance=True,
                                             averaging=train_options['averaging'],
                                             groups=None)

Fold 1 started at Sun Sep 22 22:45:06 2019
Train on 472432 samples, validate on 118108 samples
Epoch 1/130
472432/472432 [==============================] - 2s 4us/step - loss: 0.2625 - val_loss: 0.0996
Epoch 2/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.1365 - val_loss: 0.0598
Epoch 3/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.1120 - val_loss: 0.0524
Epoch 4/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.1067 - val_loss: 0.0516
Epoch 5/130
472432/472432 [==============================] - 2s 4us/step - loss: 0.1026 - val_loss: 0.0517
Epoch 6/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0993 - val_loss: 0.0520
Epoch 7/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0975 - val_loss: 0.0528
Epoch 8/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0960 - val_loss: 0.0536
Epoch 9/130
472432/472432 [======================

472432/472432 [==============================] - 2s 3us/step - loss: 0.0756 - val_loss: 0.0497
Epoch 77/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0762 - val_loss: 0.0504
Epoch 78/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0759 - val_loss: 0.0501
Epoch 79/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0754 - val_loss: 0.0494
Epoch 80/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0763 - val_loss: 0.0501
Epoch 81/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0758 - val_loss: 0.0488
Epoch 82/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0758 - val_loss: 0.0495
Epoch 83/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0757 - val_loss: 0.0492
Epoch 84/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0753 - val_loss: 0.0504
Epoch 85/130
472432/472432 [=============

472432/472432 [==============================] - 1s 3us/step - loss: 0.0820 - val_loss: 0.0636
Epoch 21/130
472432/472432 [==============================] - 1s 3us/step - loss: 0.0818 - val_loss: 0.0634
Epoch 22/130
472432/472432 [==============================] - 1s 3us/step - loss: 0.0810 - val_loss: 0.0637
Epoch 23/130
472432/472432 [==============================] - 1s 3us/step - loss: 0.0810 - val_loss: 0.0634
Epoch 24/130
472432/472432 [==============================] - 1s 3us/step - loss: 0.0806 - val_loss: 0.0629
Epoch 25/130
472432/472432 [==============================] - 1s 3us/step - loss: 0.0800 - val_loss: 0.0628
Epoch 26/130
472432/472432 [==============================] - 1s 3us/step - loss: 0.0801 - val_loss: 0.0627
Epoch 27/130
472432/472432 [==============================] - 1s 3us/step - loss: 0.0794 - val_loss: 0.0626
Epoch 28/130
472432/472432 [==============================] - 1s 3us/step - loss: 0.0795 - val_loss: 0.0622
Epoch 29/130
472432/472432 [=============

Epoch 96/130
472432/472432 [==============================] - 2s 4us/step - loss: 0.0697 - val_loss: 0.0587
Epoch 97/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0697 - val_loss: 0.0581
Epoch 98/130
472432/472432 [==============================] - 2s 4us/step - loss: 0.0693 - val_loss: 0.0588
Epoch 99/130
472432/472432 [==============================] - 2s 4us/step - loss: 0.0698 - val_loss: 0.0584
Epoch 100/130
472432/472432 [==============================] - 2s 4us/step - loss: 0.0695 - val_loss: 0.0583
Epoch 101/130
472432/472432 [==============================] - 2s 4us/step - loss: 0.0700 - val_loss: 0.0585
Epoch 102/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0702 - val_loss: 0.0584
Epoch 103/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0695 - val_loss: 0.0587
Epoch 104/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0694 - val_loss: 0.0583
Epoch 105/130
472432/47

472432/472432 [==============================] - 2s 3us/step - loss: 0.0777 - val_loss: 0.0571
Epoch 41/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0779 - val_loss: 0.0570
Epoch 42/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0767 - val_loss: 0.0566
Epoch 43/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0777 - val_loss: 0.0566
Epoch 44/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0771 - val_loss: 0.0566
Epoch 45/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0765 - val_loss: 0.0564
Epoch 46/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0765 - val_loss: 0.0566
Epoch 47/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0767 - val_loss: 0.0561
Epoch 48/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0761 - val_loss: 0.0563
Epoch 49/130
472432/472432 [=============

Epoch 116/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0695 - val_loss: 0.0543
Epoch 117/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0699 - val_loss: 0.0540
Epoch 118/130
472432/472432 [==============================] - 1s 3us/step - loss: 0.0700 - val_loss: 0.0542
Epoch 119/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0705 - val_loss: 0.0544
Epoch 120/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0703 - val_loss: 0.0543
Epoch 121/130
472432/472432 [==============================] - 2s 4us/step - loss: 0.0698 - val_loss: 0.0544
Epoch 122/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0699 - val_loss: 0.0544
Epoch 123/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0699 - val_loss: 0.0537
Epoch 124/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0701 - val_loss: 0.0540
Epoch 125/130
47243

472432/472432 [==============================] - 1s 3us/step - loss: 0.0748 - val_loss: 0.0554
Epoch 61/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0738 - val_loss: 0.0548
Epoch 62/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0735 - val_loss: 0.0546
Epoch 63/130
472432/472432 [==============================] - 1s 3us/step - loss: 0.0735 - val_loss: 0.0545
Epoch 64/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0735 - val_loss: 0.0548
Epoch 65/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0739 - val_loss: 0.0546
Epoch 66/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0734 - val_loss: 0.0545
Epoch 67/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0729 - val_loss: 0.0549
Epoch 68/130
472432/472432 [==============================] - 1s 3us/step - loss: 0.0730 - val_loss: 0.0544
Epoch 69/130
472432/472432 [=============

472432/472432 [==============================] - 1s 3us/step - loss: 0.1042 - val_loss: 0.0667
Epoch 5/130
472432/472432 [==============================] - 1s 3us/step - loss: 0.0996 - val_loss: 0.0631
Epoch 6/130
472432/472432 [==============================] - 1s 3us/step - loss: 0.0978 - val_loss: 0.0614
Epoch 7/130
472432/472432 [==============================] - 1s 3us/step - loss: 0.0958 - val_loss: 0.0605
Epoch 8/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0953 - val_loss: 0.0599
Epoch 9/130
472432/472432 [==============================] - 1s 3us/step - loss: 0.0938 - val_loss: 0.0594
Epoch 10/130
472432/472432 [==============================] - 1s 3us/step - loss: 0.0917 - val_loss: 0.0590
Epoch 11/130
472432/472432 [==============================] - 1s 3us/step - loss: 0.0913 - val_loss: 0.0588
Epoch 12/130
472432/472432 [==============================] - 1s 3us/step - loss: 0.0906 - val_loss: 0.0585
Epoch 13/130
472432/472432 [==================

472432/472432 [==============================] - 2s 3us/step - loss: 0.0744 - val_loss: 0.0539
Epoch 81/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0743 - val_loss: 0.0530
Epoch 82/130
472432/472432 [==============================] - 1s 3us/step - loss: 0.0747 - val_loss: 0.0534
Epoch 83/130
472432/472432 [==============================] - 1s 3us/step - loss: 0.0742 - val_loss: 0.0534
Epoch 84/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0736 - val_loss: 0.0533
Epoch 85/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0739 - val_loss: 0.0532
Epoch 86/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0736 - val_loss: 0.0536
Epoch 87/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0731 - val_loss: 0.0531
Epoch 88/130
472432/472432 [==============================] - 2s 3us/step - loss: 0.0734 - val_loss: 0.0530
Epoch 89/130
472432/472432 [=============

In [24]:
import pickle
pickle.dump(result_dict_keras, open(f'{p.working_folder}/results_dict_keras.pkl','wb'))

In [25]:
# test = test.sort_values('Date')
test['prediction'] = result_dict_keras['prediction']

In [26]:
# folder_path = './data/'
sub = pd.read_csv(f'{data_path}/sample_submission.csv')
sub['isFraud'] = pd.merge(sub, test['prediction'], on='TransactionID')['prediction']
sub.to_csv(f'{p.working_folder}/submission_dynamic_20_keras.csv', index=False)